# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=False)
import itertools
from scipy.stats import mannwhitneyu
import matplotlib.pyplot as plt
from tqdm import tqdm
from statannotations.Annotator import Annotator
import functools
from src.models.tabular.base import get_model_framework_dict
from glob import glob
from omegaconf import OmegaConf
import os
from pathlib import Path
from src.models.tabular.widedeep.ft_transformer import WDFTTransformerModel
from src.models.tabular.widedeep.tab_mlp import WDTabMLPModel
import xgboost as xgb
import torch
import lightgbm as lgb
from catboost import CatBoost
import pickle
from src.tasks.metrics import get_reg_metrics
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
from matplotlib import patheffects as pe

def conjunction(conditions):
    return functools.reduce(np.logical_and, conditions)


def disjunction(conditions):
    return functools.reduce(np.logical_or, conditions)

# Load data

In [ ]:
path = f"D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN/special/060_EpiSImAge"
path_save = f"{path}/figures"
Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

unn_data_type = 'harm'

feats_imm_fimmu = pd.read_excel(f"D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN/data/immuno/models/SImAge/feats_con_top10.xlsx", index_col=0).index.values

n_epi_feats = 100
feats_epi_dict = {}
feats_epi_all = set()
for feat_imm in feats_imm_fimmu:
    feats_epi_dict[feat_imm] = pd.read_excel(f"{path}/fimmu_features/{unn_data_type}/{feat_imm}/feats_con_{n_epi_feats}.xlsx", index_col=0).index.values
    feats_epi_all.update(feats_epi_dict[feat_imm])
feats_epi_all = list(feats_epi_all)

df = pd.read_excel(f"{path}/results/{unn_data_type}/table.xlsx", index_col=0)

ids_groups = {
    'GSEUNN\nControls': df.index[(df['GSE'] == 'GSEUNN') & (df['Status'] == 'Control')].values,
    'GSEUNN\nESRD': df.index[(df['GSE'] == 'GSEUNN') & (df['Status'] == 'ESRD')].values,
    'GSE87571': df.index[(df['GSE'] == 'GSE87571')].values,
    'GSE40279': df.index[(df['GSE'] == 'GSE40279')].values,
    'GSE179325\nControls': df.index[(df['GSE'] == 'GSE179325') & (df['Status'] == 'Control')].values,
    'GSE179325\nCOVID-19\nSevere': df.index[(df['GSE'] == 'GSE179325') & (df['Status'] == 'COVID-19 Severe')].values,
    'GSE179325\nCOVID-19\nMild': df.index[(df['GSE'] == 'GSE179325') & (df['Status'] == 'COVID-19 Mild')].values,
    'GSE217633\nControls': df.index[(df['GSE'] == 'GSE217633') & (df['Status'] == 'Control')].values,
    'GSE217633\nPre ART': df.index[(df['GSE'] == 'GSE217633') & (df['Status'] == 'Pre ART')].values,
    'GSE217633\nPost ART': df.index[(df['GSE'] == 'GSE217633') & (df['Status'] == 'Post ART')].values,
    'GSE118144\nControls': df.index[(df['GSE'] == 'GSE118144') & (df['Status'] == 'Control')].values,
    'GSE118144\nSLE': df.index[(df['GSE'] == 'GSE118144') & (df['Status'] == 'SLE')].values,
    'GSE42861\nControls': df.index[(df['GSE'] == 'GSE42861') & (df['Status'] == 'Control')].values,
    'GSE42861\nRheumatoid\nArthritis': df.index[(df['GSE'] == 'GSE42861') & (df['Status'] == 'Rheumatoid Arthritis')].values,
    'GSE106648\nControls': df.index[(df['GSE'] == 'GSE106648') & (df['Status'] == 'Control')].values,
    'GSE106648\nMultiple\nSclerosis': df.index[(df['GSE'] == 'GSE106648') & (df['Status'] == 'Multiple Sclerosis')].values,
    'GSE67530\nControls': df.index[(df['GSE'] == 'GSE67530') & (df['Status'] == 'Control')].values,
    'GSE67530\nICU': df.index[(df['GSE'] == 'GSE67530') & (df['Status'] == 'ICU')].values,
    'GSE67530\nICU ARDS': df.index[(df['GSE'] == 'GSE67530') & (df['Status'] == 'ICU ARDS')].values,
}

pairs_groups = [
    ('GSEUNN\nControls', 'GSEUNN\nESRD'),
    ('GSEUNN\nControls', 'GSE87571'),
    ('GSEUNN\nControls', 'GSE40279'),
    ('GSE87571', 'GSE40279'),
    ('GSEUNN\nControls', 'GSE179325\nControls'),
    ('GSE179325\nControls', 'GSE179325\nCOVID-19\nMild'),
    ('GSE179325\nCOVID-19\nMild', 'GSE179325\nCOVID-19\nSevere'),
    ('GSE179325\nControls', 'GSE179325\nCOVID-19\nSevere'),
    ('GSEUNN\nControls', 'GSE217633\nControls'),
    ('GSE217633\nControls', 'GSE217633\nPre ART'),
    ('GSE217633\nPre ART', 'GSE217633\nPost ART'),
    ('GSE217633\nControls', 'GSE217633\nPost ART'),
    ('GSEUNN\nControls', 'GSE118144\nControls'),
    ('GSE118144\nControls', 'GSE118144\nSLE'),
    ('GSEUNN\nControls', 'GSE42861\nControls'),
    ('GSE42861\nControls', 'GSE42861\nRheumatoid\nArthritis'),
    ('GSEUNN\nControls', 'GSE106648\nControls'),
    ('GSE106648\nControls', 'GSE106648\nMultiple\nSclerosis'),
    ('GSEUNN\nControls', 'GSE67530\nControls'),
    ('GSE67530\nControls', 'GSE67530\nICU'),
    ('GSE67530\nControls', 'GSE67530\nICU ARDS'),
    ('GSE67530\nICU', 'GSE67530\nICU ARDS'),
]

colors_groups = {
    'GSEUNN\nControls': 'crimson',
    'GSEUNN\nESRD': 'dodgerblue',
    'GSE87571': 'limegreen',
    'GSE40279': 'gold',
    'GSE179325\nControls': 'olive',
    'GSE179325\nCOVID-19\nMild': 'burlywood',
    'GSE179325\nCOVID-19\nSevere': 'darkorchid',
    'GSE217633\nControls': 'slateblue',
    'GSE217633\nPre ART': 'yellow',
    'GSE217633\nPost ART': 'orange',
    'GSE118144\nControls': 'aqua',
    'GSE118144\nSLE': 'darkkhaki',
    'GSE42861\nControls': 'coral',
    'GSE42861\nRheumatoid\nArthritis': 'brown',
    'GSE106648\nControls': 'palegreen',
    'GSE106648\nMultiple\nSclerosis': 'lightsteelblue',
    'GSE67530\nControls': 'maroon',
    'GSE67530\nICU': 'aqua',
    'GSE67530\nICU ARDS': 'orchid',
}

colors_gses_bckg = {
    'GSEUNN': 'aliceblue',
    'GSE87571': 'whitesmoke',
    'GSE40279': 'cornsilk',
    'GSE179325': 'ghostwhite',
    'GSE217633': 'lavenderblush',
    'GSE118144': 'linen',
    'GSE42861': 'mintcream',
    'GSE106648': 'oldlace',
    'GSE67530': 'snow',
}

colors_groups_bckg = {
    'GSEUNN\nControls': colors_gses_bckg['GSEUNN'],
    'GSEUNN\nESRD': colors_gses_bckg['GSEUNN'],
    'GSE87571': colors_gses_bckg['GSE87571'],
    'GSE40279': colors_gses_bckg['GSE40279'],
    'GSE179325\nControls': colors_gses_bckg['GSE179325'],
    'GSE179325\nCOVID-19\nMild': colors_gses_bckg['GSE179325'],
    'GSE179325\nCOVID-19\nSevere': colors_gses_bckg['GSE179325'],
    'GSE217633\nControls': colors_gses_bckg['GSE217633'],
    'GSE217633\nPre ART': colors_gses_bckg['GSE217633'],
    'GSE217633\nPost ART': colors_gses_bckg['GSE217633'],
    'GSE118144\nControls': colors_gses_bckg['GSE118144'],
    'GSE118144\nSLE': colors_gses_bckg['GSE118144'],
    'GSE42861\nControls': colors_gses_bckg['GSE42861'],
    'GSE42861\nRheumatoid\nArthritis': colors_gses_bckg['GSE42861'],
    'GSE106648\nControls': colors_gses_bckg['GSE106648'],
    'GSE106648\nMultiple\nSclerosis': colors_gses_bckg['GSE106648'],
    'GSE67530\nControls': colors_gses_bckg['GSE67530'],
    'GSE67530\nICU': colors_gses_bckg['GSE67530'],
    'GSE67530\nICU ARDS': colors_gses_bckg['GSE67530'],
}

ages = [
    'EpiSImAge',
    'Hannum',
    'PC-Hannum',
    'Horvath',
    'PC-Horvath',
    'SkinBloodAge',
    'PC-SkinBloodAge',
    'PhenoAge',
    'PC-PhenoAge',
    'GrimAge',
    'PC-GrimAge',
    'GrimAge2',
]

df_mw = pd.DataFrame(index=ages, columns=[f"{pair[0]}\nVS\n{pair[1]}" for pair in pairs_groups])
df_mae = pd.DataFrame(index=ages, columns=list(ids_groups.keys()))
df_pearsonr = pd.DataFrame(index=ages, columns=list(ids_groups.keys()))
df_maa = pd.DataFrame(index=ages, columns=list(ids_groups.keys()))
for age_type in ages:
    for pair in pairs_groups:
        stat, pval = mannwhitneyu(
            df.loc[ids_groups[pair[0]], f"{age_type} acceleration"].values,
            df.loc[ids_groups[pair[1]], f"{age_type} acceleration"].values,
            alternative='two-sided'
        )
        df_mw.at[age_type, f"{pair[0]}\nVS\n{pair[1]}"] = pval
    
    for group, ids_group in ids_groups.items():
        df_mae.at[age_type, group] = mean_absolute_error(
            df.loc[ids_group, "Age"].values,
            df.loc[ids_group, age_type].values,
        )
        df_pearsonr.at[age_type, group] = pearsonr(
            df.loc[ids_group, "Age"].values,
            df.loc[ids_group, age_type].values,
        ).statistic
        df_maa.at[age_type, group] = np.mean(df.loc[ids_group, f"{age_type} acceleration"].values)
df_mw.to_excel(f"{path_save}/mw.xlsx")
df_mae.to_excel(f"{path_save}/mae.xlsx")
df_pearsonr.to_excel(f"{path_save}/pearsonr.xlsx")
df_maa.to_excel(f"{path_save}/maa.xlsx")

# Plot UNN only figures

## Hists of age distributions in different datasets

In [ ]:
Path(f"{path_save}/EpiSImAge").mkdir(parents=True, exist_ok=True)

n_rows = 3
n_cols = 3
fig_height = 9
fig_width = 12
hist_bins = np.linspace(5, 115, 23)
sns.set_theme(style='whitegrid')
fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), gridspec_kw={}, sharey=False, sharex=False)
for gse_id, gse in tqdm(enumerate(colors_gses_bckg.keys())):
    row_id, col_id = divmod(gse_id, n_cols)
    histplot = sns.histplot(
        data=df.loc[df['GSE'] == gse, :],
        bins=hist_bins,
        edgecolor='k',
        linewidth=1,
        x="Age",
        hue='Group',
        palette=colors_groups,
        ax=axs[row_id, col_id]
    )
    axs[row_id, col_id].set(xlim=(0, 120))
    axs[row_id, col_id].set_facecolor(colors_gses_bckg[gse])
fig.tight_layout()    
fig.savefig(f"{path_save}/EpiSImAge/hist_age.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path_save}/EpiSImAge/hist_age.pdf", bbox_inches='tight')
plt.close(fig)

## Scatters and violins for EpiSImAge

In [ ]:
Path(f"{path_save}/EpiSImAge").mkdir(parents=True, exist_ok=True)

n_rows = 2
n_cols = 10
fig_height = 9
fig_width = 32

sns.set_theme(style='whitegrid')
fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), gridspec_kw={}, sharey=True, sharex=False)

for group_id, group in tqdm(enumerate(colors_groups.keys())):
    row_id, col_id = divmod(group_id, n_cols)
    scatter = sns.scatterplot(
        data=df.loc[ids_groups[group]],
        x="Age",
        y="EpiSImAge",
        linewidth=0.2,
        alpha=0.75,
        edgecolor="k",
        s=40,
        color=colors_groups[group],
        ax=axs[row_id, col_id],
    )
    axs[row_id, col_id].set_title(f"{group}\n(MAE: {df_mae.at['EpiSImAge', group]:0.1f})")
    bisect = sns.lineplot(
        x=[0, 120],
        y=[0, 120],
        linestyle='--',
        color='black',
        linewidth=1.0,
        ax=axs[row_id, col_id]
    )
    axs[row_id, col_id].set_xlim(0, 120)
    axs[row_id, col_id].set_ylim(0, 120)
    axs[row_id, col_id].set_facecolor(colors_groups_bckg[group])
    if row_id == 0 and col_id != n_cols - 1:
        axs[row_id, col_id].set_xticklabels([])
        axs[row_id, col_id].set_xlabel('')
axs[n_rows - 1, n_cols - 1].axis('off')
fig.tight_layout()    
fig.savefig(f"{path_save}/EpiSImAge/scatters.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path_save}/EpiSImAge/scatters.pdf", bbox_inches='tight')
plt.close(fig)

sns.set_theme(style='whitegrid')
fig, ax = plt.subplots(figsize=(32, 6))

for group_id, group in enumerate(colors_groups):
    ax.axvspan(group_id - 0.5, group_id + 0.5, facecolor=colors_groups_bckg[group], alpha=1.0, zorder=0, lw=0)
sns.violinplot(
    data=df,
    x='Group',
    y='EpiSImAge acceleration',
    palette=colors_groups,
    scale='width',
    order=list(colors_groups.keys()),
    saturation=0.75,
    ax=ax
)
ax.set_xlabel(f"")

pval_formatted = [f'{pval:.1e}' for pval in df_mw.loc['EpiSImAge', :].values]
annotator = Annotator(
    ax,
    pairs=pairs_groups,
    data=df,
    x='Group',
    y='EpiSImAge acceleration',
    order=list(colors_groups),
)
annotator.set_custom_annotations(pval_formatted)
annotator.configure(loc='outside')
annotator.annotate()
fig.savefig(f"{path_save}/EpiSImAge/violins.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path_save}/EpiSImAge/violins.pdf", bbox_inches='tight')
plt.close(fig)

# Comparison of EpiSImAge with other EpiAges

## Scatters

In [ ]:
Path(f"{path_save}/Comparison").mkdir(parents=True, exist_ok=True)

n_rows = len(ages)
n_cols = len(ids_groups)
fig_height = 32
fig_width = 42

sns.set_theme(style='whitegrid')
fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), gridspec_kw={}, sharey=True, sharex=True)

for group_id, group in tqdm(enumerate(colors_groups.keys())):
    for age_id, age_type in enumerate(ages):
        kdeplot = sns.kdeplot(
            data=df.loc[ids_groups[group]],
            x='Age',
            y=age_type,
            fill=True,
            cbar=False,
            color=colors_groups[group],
            cut=0,
            legend=False,
            ax=axs[age_id, group_id]
        )
        regplot = sns.regplot(
            data=df.loc[ids_groups[group]],
            x='Age',
            y=age_type,
            scatter=False,
            color='black',
            truncate=True,
            ax=axs[age_id, group_id]
        )
        bisect = sns.lineplot(
            x=[0, 120],
            y=[0, 120],
            linestyle='--',
            color='black',
            linewidth=1.0,
            ax=axs[age_id, group_id]
        )
        axs[age_id, group_id].set_xlim(0, 120)
        axs[age_id, group_id].set_ylim(0, 120)
        axs[age_id, group_id].set_facecolor(colors_groups_bckg[group])
        if age_id > 0:
            axs[age_id, group_id].set_title(f"MAE: {df_mae.at[age_type, group]:0.1f}\nPearson: {df_pearsonr.at[age_type, group]:0.2f}\nMean Acceleration: {df_maa.at[age_type, group]:0.1f}")
        else:
            axs[age_id, group_id].set_title(f"{group}\n\nMAE: {df_mae.at[age_type, group]:0.1f}\nPearson: {df_pearsonr.at[age_type, group]:0.2f}\nMean Acceleration: {df_maa.at[age_type, group]:0.1f}")
fig.tight_layout()    
fig.savefig(f"{path_save}/Comparison/scatters.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path_save}/Comparison/scatters.pdf", bbox_inches='tight')
plt.close(fig)

## Clustermaps

In [ ]:
df_fig = pd.read_excel(f"{path_save}/maa.xlsx", index_col=0)
cols_rename = {x: x.replace('\n', ' ') for x in df_fig.columns}
cols_rename_inv = {v: k for k, v in cols_rename.items()}
df_fig.rename(columns=cols_rename, inplace=True)
sns.set_theme(style='whitegrid')
clustermap = sns.clustermap(
    df_fig,
    annot=True,
    col_cluster=False,
    row_cluster=True,
    fmt=".1f",
    center=0.0,
    cmap='seismic',
    linewidth=0.1,
    linecolor='black',
    tree_kws=dict(linewidths=1.5),
    figsize=(12, 8),
    cbar_kws={'orientation': 'horizontal'}
)
clustermap.ax_heatmap.set_xlabel('')
clustermap.ax_heatmap.set_ylabel('')
clustermap.ax_cbar.tick_params()
for spine in clustermap.ax_cbar.spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
clustermap.ax_heatmap.set_xticklabels(clustermap.ax_heatmap.get_xmajorticklabels(), path_effects=[pe.withStroke(linewidth=0.75, foreground="black")])
for tick_label in clustermap.ax_heatmap.get_xticklabels():
    tick_label.set_color(colors_groups[cols_rename_inv[tick_label.get_text()]])
x0, _y0, _w, _h = clustermap.cbar_pos
clustermap_pos = clustermap.ax_heatmap.get_position()
clustermap.ax_cbar.set_position([clustermap_pos.x0, clustermap_pos.y1 + 0.05, clustermap_pos.width, 0.03])
clustermap.ax_cbar.set_title(r"$\langle$ Acceleration $\rangle$")
clustermap.ax_cbar.tick_params()
for spine in clustermap.ax_cbar.spines:
    clustermap.ax_cbar.spines[spine].set_linewidth(1)
plt.savefig(f"{path_save}/Comparison/clustermap_maa.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/Comparison/clustermap_maa.pdf", bbox_inches='tight')
plt.close(clustermap.fig)

df_fig = pd.read_excel(f"{path_save}/mae.xlsx", index_col=0)
cols_rename = {x: x.replace('\n', ' ') for x in df_fig.columns}
cols_rename_inv = {v: k for k, v in cols_rename.items()}
df_fig.rename(columns=cols_rename, inplace=True)
sns.set_theme(style='whitegrid')
clustermap = sns.clustermap(
    df_fig,
    annot=True,
    col_cluster=False,
    row_cluster=True,
    fmt=".1f",
    cmap='flare',
    linewidth=0.1,
    linecolor='black',
    tree_kws=dict(linewidths=1.5),
    figsize=(12, 8),
    cbar_kws={'orientation': 'horizontal'}
)
clustermap.ax_heatmap.set_xlabel('')
clustermap.ax_heatmap.set_ylabel('')
for spine in clustermap.ax_cbar.spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
clustermap.ax_heatmap.set_xticklabels(clustermap.ax_heatmap.get_xmajorticklabels(), path_effects=[pe.withStroke(linewidth=0.75, foreground="black")])
for tick_label in clustermap.ax_heatmap.get_xticklabels():
    tick_label.set_color(colors_groups[cols_rename_inv[tick_label.get_text()]])
x0, _y0, _w, _h = clustermap.cbar_pos
clustermap_pos = clustermap.ax_heatmap.get_position()
clustermap.ax_cbar.set_position([clustermap_pos.x0, clustermap_pos.y1 + 0.05, clustermap_pos.width, 0.03])
clustermap.ax_cbar.set_title("MAE")
clustermap.ax_cbar.tick_params()
for spine in clustermap.ax_cbar.spines:
    clustermap.ax_cbar.spines[spine].set_linewidth(1)
plt.savefig(f"{path_save}/Comparison/clustermap_mae.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/Comparison/clustermap_mae.pdf", bbox_inches='tight')
plt.close(clustermap.fig)

df_fig = pd.read_excel(f"{path_save}/pearsonr.xlsx", index_col=0)
cols_rename = {x: x.replace('\n', ' ') for x in df_fig.columns}
cols_rename_inv = {v: k for k, v in cols_rename.items()}
df_fig.rename(columns=cols_rename, inplace=True)
sns.set_theme(style='whitegrid')
clustermap = sns.clustermap(
    df_fig,
    annot=True,
    col_cluster=False,
    row_cluster=True,
    fmt=".2f",
    cmap='hot',
    linewidth=0.1,
    linecolor='black',
    tree_kws=dict(linewidths=1.5),
    figsize=(12, 8),
    cbar_kws={'orientation': 'horizontal'}
)
clustermap.ax_heatmap.set_xlabel('')
clustermap.ax_heatmap.set_ylabel('')
for spine in clustermap.ax_cbar.spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
clustermap.ax_heatmap.set_xticklabels(clustermap.ax_heatmap.get_xmajorticklabels(), path_effects=[pe.withStroke(linewidth=0.75, foreground="black")])
for tick_label in clustermap.ax_heatmap.get_xticklabels():
    tick_label.set_color(colors_groups[cols_rename_inv[tick_label.get_text()]])
x0, _y0, _w, _h = clustermap.cbar_pos
clustermap_pos = clustermap.ax_heatmap.get_position()
clustermap.ax_cbar.set_position([clustermap_pos.x0, clustermap_pos.y1 + 0.05, clustermap_pos.width, 0.03])
clustermap.ax_cbar.set_title(r"Pearson $\rho$")
clustermap.ax_cbar.tick_params()
for spine in clustermap.ax_cbar.spines:
    clustermap.ax_cbar.spines[spine].set_linewidth(1)
plt.savefig(f"{path_save}/Comparison/clustermap_pearsonr.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/Comparison/clustermap_pearsonr.pdf", bbox_inches='tight')
plt.close(clustermap.fig)

df_fig = pd.read_excel(f"{path_save}/mw.xlsx", index_col=0)
cols_rename = {x: x.replace('\n', ' ') for x in df_fig.columns}
df_fig.rename(columns=cols_rename, inplace=True)
for col in df_fig.columns:
    df_fig[col] = -np.log10(df_fig[col])
sns.set_theme(style='whitegrid')
cmap = plt.get_cmap("cool").copy()
cmap.set_under('black')
clustermap = sns.clustermap(
    df_fig,
    annot=True,
    col_cluster=False,
    row_cluster=True,
    fmt=".1f",
    cmap=cmap,
    vmin=-np.log(0.05),
    linewidth=0.1,
    linecolor='black',
    tree_kws=dict(linewidths=1.5),
    figsize=(16, 10),
    cbar_kws={'orientation': 'horizontal'}
)
clustermap.ax_heatmap.set_xlabel('')
clustermap.ax_heatmap.set_ylabel('')
for spine in clustermap.ax_cbar.spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
x0, _y0, _w, _h = clustermap.cbar_pos
clustermap_pos = clustermap.ax_heatmap.get_position()
clustermap.ax_cbar.set_position([clustermap_pos.x0, clustermap_pos.y1 + 0.05, clustermap_pos.width, 0.03])
clustermap.ax_cbar.set_title(r"$-\log_{10}$(Mann-Whitney U test p-value)")
clustermap.ax_cbar.tick_params()
for spine in clustermap.ax_cbar.spines:
    clustermap.ax_cbar.spines[spine].set_linewidth(1)
plt.savefig(f"{path_save}/Comparison/clustermap_mw.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/Comparison/clustermap_mw.pdf", bbox_inches='tight')
plt.close(clustermap.fig)